In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
from PIL import Image
from torchvision.transforms import ToTensor, Resize, Grayscale
from torchvision.io import read_image
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from torchvision import models, transforms
from PIL import Image


In [14]:
# df = pd.read_excel(r'C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\Gender_2000.xlsx')
# df["Images"] = df["Images"].replace(r"\\", "/", regex=True)
# df["Images"] = df["Images"].replace(r"data/img_align_celeba", "C:/Users/busse/Bachelorarbeit/CICD-Pipeline-Gender-Recognition/data/img_align_celeba/", regex=True)
# df.to_excel("Gender_2000.xlsx", index=False)

In [15]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=2)
        self.fc1 = nn.Linear(256 * 6 * 6, 256)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 256 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def extract_features(image_paths):
        # Load the pre-trained model
        model = models.resnet50(pretrained=True)

        # Remove the last layer (the classifier)
        model = torch.nn.Sequential(*(list(model.children())[:-1]))

        # Define the image transformations
        transform = transforms.Compose([
            transforms.Resize((178, 218)),
            transforms.Grayscale(num_output_channels=3),  # Convert to RGB
            transforms.ToTensor(),
        ])

        features = []
        for image_path in image_paths:
            # Load and transform the image
            img = Image.open(image_path)
            img = transform(img).unsqueeze(0)  # Add a batch dimension

            # Extract features
            with torch.no_grad():
                feature = model(img)

            features.append(feature)

        features = torch.stack(features)
        return features



In [17]:
n=2000
path = r'C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\Gender_2000.xlsx'
print(path)
df = pd.read_excel(path)
df_sample = df.groupby('Gender', group_keys=False).apply(lambda x: x.sample(n=n, random_state=42))
df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)
df_sample.to_excel(fr'C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\excel_sheets\Gender_{n}.xlsx', index=False)
X = CNN.extract_features(df_sample['Images'])
X = X / 255.0
y_gender = np.array(df_sample['Gender'])


C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\Gender_2000.xlsx


c:\Users\busse\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\busse\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

path = r'C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\Gender_2000.xlsx'

class CustomImageDataset(Dataset):
    def __init__(self, excel_file, transform=None):
        self.data = pd.read_excel(excel_file)
        self.df = pd.DataFrame(self.data)
        self.transform = transform
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 output units for 2 classes
        print(self.data.head())
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.data.head()
        img_path = self.df["Images"]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.data.iloc[idx, 1]  # adjust this if your label is not in the second column
        return image, label

# Define your transformations
transform = transforms.Compose([
    transforms.Resize((178, 218)),
    transforms.ToTensor(),
    transforms.Grayscale(),
])

# Create the dataset
dataset = CustomImageDataset(path, transform=transform)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=32)

NameError: name 'n' is not defined

In [ ]:

path = fr'C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\excel_sheets\Gender_{n}.xlsx'
read = pd.read_excel(path)
df['Gender'] = df['Gender'].replace(0, -1)
df.to_excel(path)

NameError: name 'n' is not defined

In [18]:
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
net = CNN()

criterion = nn.CrossEntropyLoss()

path = fr'C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\model\excel_sheets\Gender_{n}.xlsx'

class CustomImageDataset(Dataset):
  def __init__(self, excel_file, transform=None):
    self.data = pd.read_excel(excel_file)
    self.df = pd.DataFrame(self.data)
    self.transform = transform
    self.fc1 = nn.Linear(512, 128)
    # self.fc2 = nn.Linear(128, 2)  # 2 output units for 2 classes
    self.data = self.data.drop("Unnamed: 0", axis=1)
    print(self.data.head())

    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img_path = self.df.loc[idx, "Images"]  # Get the image path for the given index
    image = Image.open(img_path).convert('RGB')
    if self.transform:
      image = self.transform(image)
    label = self.data.iloc[idx, 1]  # adjust this if your label is not in the second column
    return image, label

# Define your transformations
transform = transforms.Compose([
  transforms.Resize((178, 218)),
  transforms.ToTensor(),
  transforms.Grayscale(),
])

# Create the dataset
dataset = CustomImageDataset(path, transform=transform)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=32)
# outputs = net(inputs)
# print(outputs)
# loss = criterion(outputs, labels)


KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
for epoch in range(100):  # 100 epochs
    for i,(inputs, labels) in dataloader:  # Replace with your DataLoader
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        # loss.backward()
        optimizer.step()
    # # print statistics
    # running_loss += loss.item()
    # print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
    # running_loss = 0.0

KeyboardInterrupt: 